### Assignment - IO

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import linearmodels as ln
from statsmodels.sandbox.regression.gmm import IV2SLS
from scipy.optimize import minimize, fsolve, root, least_squares
from scipy import ndimage
from pystout import pystout
from tabulate import tabulate
from scipy.stats import uniform as u
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("autodata.csv")
df.rename(columns = {'Firm ID (there is a different number for each firm)':'Firm'}, inplace = True)
df.rename(columns = {'Price($1000) (list price)':'Price'}, inplace = True)

In [3]:
#Calculate shares
s0 = 0.6 #Set outside good share
df['Total q'] = df['Quantity'].sum()
df['Share'] = df['Quantity'] / df['Total q'] * (1 - s0)

In [4]:
#And sort data
df.sort_values(by=['Price'], ascending = False, inplace=True) #Sort dataset
df.reset_index(inplace = True)

### Demand side estimation

In [5]:
#Calculate thresholds
sN = df['Share'].iloc[0]
#General formatof inverse of Uniform cdf: F(p) = a + p(b - a)
#If we assume a = 0 and b = 1
#We will always maintain a = 0 (idea: lowest possible taste for quality is not caring)
b = 10
df['Δ'] = u.ppf(1 - sN)

for i, r in df.iloc[1:].iterrows():
    Δ_previous = df['Δ'].iloc[i-1]
    s = df['Share'].iloc[i]
    
    df['Δ'].iloc[i] = u.ppf(u.cdf(Δ_previous) - s)

C:\Users\djhen\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [6]:
#Resort in opposite way to get quality
df.sort_values(by=['Price'], inplace=True) #Sort dataset
df.reset_index(inplace = True)

In [7]:
#Calculate implied quality levels
df['δ'] = 0

for i, r in df.iterrows():
    
    if i == 0:
        δ_previous = 0
        p_previous = 0
        
    else:
        δ_previous = df['δ'].iloc[i-1]
        p_previous = df['Price'].iloc[i-1]
        
    p = df['Price'].iloc[i]
    Δ = df['Δ'].iloc[i]
    
    df['δ'].iloc[i] = δ_previous + (p - p_previous) / Δ

In [8]:
# Run OLS regression
x = df[['Air Conditioning', 'Weight of Car', 'Horsepower']]
y = df['δ']
model1 = sm.OLS(y, x).fit()
model1_α = model1.params

In [9]:
pystout(models=[model1],
        file='OLS_demand.tex',
        digits=3,
        endog_names=False,
        exogvars=['Air Conditioning', 'Weight of Car', 'Horsepower'],
        stars=False,
        modstat={'nobs':'Obs','rsquared_adj':'Adj. R\sym{2}'}
        )

### Supply side estimation

In [10]:
#Estimate parameters under marginal cost pricing
x = df[['Air Conditioning', 'Weight of Car', 'Horsepower', 'Quantity']]
y = df['Price']
supply1 = sm.OLS(y, x).fit()

In [11]:
#Estimate own and cross-price elasticities for every good
J = df.shape[0]
η = np.zeros((J, J))
own_price = np.zeros(J)

#Own-price
for i in range(J):
    p = df['Price'].iloc[i]
    δ = df['δ'].iloc[i]
    Δ = df['Δ'].iloc[i]
    
    if i == 0:
        δ_lower = 0
        δ_higher = df['δ'].iloc[i+1]
        Δ_higher = df['Δ'].iloc[i+1]
        
        η[i,i] = - (u.pdf(Δ_higher) / (δ_higher - δ) + u.pdf(Δ) / (δ - δ_lower) )
        own_price[i] = - (u.pdf(Δ_higher) / (δ_higher - δ) + u.pdf(Δ) / (δ - δ_lower) )
    
    elif 0 < i < J - 1 :
        δ_lower = df['δ'].iloc[i-1]
        δ_higher = df['δ'].iloc[i+1]
        Δ_higher = df['Δ'].iloc[i+1]
        
        η[i,i] = - (u.pdf(Δ_higher) / (δ_higher - δ) + u.pdf(Δ) / (δ - δ_lower) )
        own_price[i] = - (u.pdf(Δ_higher) / (δ_higher - δ) + u.pdf(Δ) / (δ - δ_lower) )
        

    elif i == J - 1:
        δ_lower = df['δ'].iloc[i-1]
        δ_higher = float('inf')
        Δ_higher = float('inf')
        
        η[i,i] = - (u.pdf(Δ_higher) / (δ_higher - δ) + u.pdf(Δ) / (δ - δ_lower) )
        own_price[i] = - (u.pdf(Δ_higher) / (δ_higher - δ) + u.pdf(Δ) / (δ - δ_lower) )
    
#Cross-price
for i in range(J):
    for j in range(J):
        if j == i + 1:
            δ = df['δ'].iloc[i]
            if i < J - 1:
                Δ_higher = df['Δ'].iloc[j]
                δ_higher = df['δ'].iloc[j]
                
                η[i,j] = u.pdf(Δ_higher) / (δ_higher - δ)
                
            elif i == J - 1:
                Δ_higher = float('inf')
                δ_higher = float('inf')
                
                η[i,j] = u.pdf(Δ_higher) / (δ_higher - δ)
            
        elif j == i - 1:
            δ = df['δ'].iloc[i]
            Δ = df['Δ'].iloc[i]
            
            if i > 0:
                δ_lower = df['δ'].iloc[j]
                
                η[i,j] = u.pdf(Δ) / (δ - δ_lower)
                
            elif i == 0:
                δ_lower = 0 
                
                η[i,j] = u.pdf(Δ) / (δ - δ_lower)

#Replace infinity values by really high numbers (otherwise the rest of the code crashes)
η[η == -np.inf] = -10000
η[η == np.inf] = 10000


<ipython-input-11-36e68a6df630>:25: RuntimeWarning: divide by zero encountered in double_scalars
  η[i,i] = - (u.pdf(Δ_higher) / (δ_higher - δ) + u.pdf(Δ) / (δ - δ_lower) )
<ipython-input-11-36e68a6df630>:26: RuntimeWarning: divide by zero encountered in double_scalars
  own_price[i] = - (u.pdf(Δ_higher) / (δ_higher - δ) + u.pdf(Δ) / (δ - δ_lower) )
<ipython-input-11-36e68a6df630>:46: RuntimeWarning: divide by zero encountered in double_scalars
  η[i,j] = u.pdf(Δ_higher) / (δ_higher - δ)
<ipython-input-11-36e68a6df630>:61: RuntimeWarning: divide by zero encountered in double_scalars
  η[i,j] = u.pdf(Δ) / (δ - δ_lower)


In [12]:
#Define ownership matrices

#(i)Single product ownership -- equivalent to marginal cost pricing
Ω_single = np.identity(J)

#(ii)Nash Bargaining -- Firms set prices
Ω_ne = np.identity(J)
for i in range(J - 1):
    if df['Firm'].iloc[i] == df['Firm'].iloc[i+1]:
        Ω_ne[i,i+1] = 1
    if df['Firm'].iloc[i] == df['Firm'].iloc[i-1]:
        Ω_ne[i,i-1] = 1

#(iii)Perfect collusion -- all products owned by the same firm
Ω_collusion = np.identity(J)
for i in range(J):
    for j in range(J):
        if i == j + 1:
            Ω_collusion[i,j] = 1
        if i == j - 1:
            Ω_collusion[i,j] = 1
            

In [13]:
#Retrieve values from data-frame for marginal cost calculations
p = df['Price'].values
s = df['Share'].values

In [14]:
#calculate marginal cost using each of the different ownership matrices
mc_single = p + np.linalg.inv(Ω_single * η) @ s
mc_collusion = p + np.linalg.inv(Ω_collusion * η) @ s
#mc_ne = p + np.linalg.inv(Ω_ne * η) @ s = 0

In [15]:
bounds = [[0, 20000]] * J
bounds

[[0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],
 [0, 20000],

In [16]:
#Marginal cost calculations do not work for nash equilibrium pricing because the matrix is singular
mc_init = np.ones(J)
    
def mcfunc_norm(mc):
    if mc.any() < 0:
        return mc_init
    if mc.all() > 0:
        return np.linalg.norm(s + (Ω_ne * η) @ (p - mc))
    
def mcfunc(mc):
    if mc.any() < 0:
        return mc_init
    if mc.all() > 0:
        return s + (Ω_ne * η) @ (p - mc)
    
obj = fsolve(mcfunc, mc_init)

results = minimize(mcfunc_norm, mc_init,
                   tol=1e-10)


In [17]:
mc_ne = results.x

In [18]:
#Run the various regressions
x = df[['Air Conditioning', 'Weight of Car', 'Horsepower', 'Quantity']]

supply_single = sm.OLS(mc_single, x).fit()
supply_ne = sm.OLS(mc_ne, x).fit()
supply_collusion = sm.OLS(mc_collusion, x).fit()
supply_ne.params

Air Conditioning   -1899.295326
Weight of Car          4.785491
Horsepower           -40.210518
Quantity               0.002215
dtype: float64

In [19]:
#### Output regression results into table
pystout(models=[supply_single, supply_ne, supply_collusion],
        file='supply_models.tex',
        digits=3,
        endog_names=False,
        exogvars=['Air Conditioning', 'Weight of Car', 'Horsepower','Quantity'],
        stars=False,
        modstat={'nobs':'Obs','rsquared_adj':'Adj. R\sym{2}'}
        )